In [154]:
import KNNclassy
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import classyRF_final as classyRF
from sklearn.neighbors import NearestNeighbors
importlib.reload(KNNclassy)

<module 'KNNclassy' from '/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/KNNclassy.py'>

In [151]:
frac_testing = 0.3
nneigh = [10]
pathClassy = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/"
pathData = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/input/"
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]
EOSgrey = ["APR4_EPP",  "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLY2", "SLY9", "SLY230A"]
EOScolor = ["BHF_BBB2", "MS1_PP", "SLy"]
color_v = ['orange', 'green', 'blue']
BayesFactor = [1.526, 1.555, 0.056, 1.422, 1.177, 1.283, 0.276, 0.001, 0.009, 0.176, 0.179, 0.159, 0.108, 0.107, 0.33, 0.025, 0.288, 0.29, 0.618, 1.0, 1.028, 0.37, 0.932]

In [152]:
EOSdic = {}
count = 0
#EOS = ['SLy']

In [145]:
for eos in EOS:
    
    print("Doing", eos)
    print('*'*60)

    for kk in nneigh:
        
        print("Nº of neighbors: ", kk)
        print('-'*40)

        KNN = KNNclassy.ClassificationKNN()
        KNN.load_train_dataset(pathData+eos+"/NS/train_NS_karoo_"+eos+"_s300_f0d7.csv",pathData+eos+"/EMB/train_EMB_karoo_"+eos+"_s300_f0d7.csv")
        KNN.load_test_dataset(pathData+eos+"/NS/test_NS_ID_LABEL_"+eos+"_s300_f0d7.csv",pathData+eos+"/EMB/test_EMB_ID_LABEL_"+eos+"_s300_f0d7.csv")
    
        KNN.loadModel(pathClassy+"optimal_models_final/", "knn_3cat_grid_eos_"+eos)
        new_dic = {}
        new_dic['knn'] = KNN
        new_dic['bayes'] = BayesFactor[count]
        EOSdic[eos] = new_dic
    
        print('*'*60)
        
    count = count + 1
        

Doing APR4_EPP
************************************************************
Nº of neighbors:  10
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  139273
Datasets loaded!
************************************************************
************************************************************
Loading testing data...
Nº of features:  5
Nº of events for testing:  59688
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final/knn_3cat_grid_eos_APR4_EPP.joblib
************************************************************
Doing BHF_BBB2
************************************************************
Nº of neighbors:  10
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of 

************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  140784
Datasets loaded!
************************************************************
************************************************************
Loading testing data...
Nº of features:  5
Nº of events for testing:  60336
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final/knn_3cat_grid_eos_SK272.joblib
************************************************************
Doing SKI2
************************************************************
Nº of neighbors:  10
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  139343
Datasets loaded!
************************************************************
**************************************

************************************************************
Loading testing data...
Nº of features:  5
Nº of events for testing:  59208
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final/knn_3cat_grid_eos_SLY230A.joblib
************************************************************


In [155]:
pathData = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/input/"
pathForest = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNNvsRF/RF_marina/data/"

i_m1r=1; i_m2r=2; i_x1r=3; i_x2r=4; i_snr=8;
recovered=[i_m1r, i_m2r, i_x1r, i_x2r, i_snr]
EOSdicRF = {}
count = 0
for eos in EOS:
    RF = classyRF.ClassificationRF()
    RF.loadPickle(pathForest+'random_forest_'+eos)
    RF.load_test_dataset(pathData+eos+'/EMB/', 'test_EMB_ID_LABEL_'+eos+'_s300_f0d7.csv')
    aux = RF.labels_test
    RF.label_REM = aux
    RF.load_test_dataset(pathData+eos+'/NS/', 'test_NS_ID_LABEL_'+eos+'_s300_f0d7.csv')
    RF.label_NS = RF.labels_test
    RF.labels_test = RF.labels_test + aux
    RF.subset_test(recovered)
    new_dic = {}
    new_dic['forest'] = RF
    new_dic['weight'] = BayesFactor[count]
    EOSdicRF[eos] = new_dic
    count = count + 1 

In [148]:
def matrix(algo,nneigh,eos):
    total_events = len(algo.label_test)
    nplus_NO = np.zeros(total_events)
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    
    for i in range(0,total_events):
        distance, closestn = algo.model.kneighbors(algo.xtest[i].reshape(1, -1),n_neighbors=nneigh,return_distance = True)
     #   print(1/distance[0])
        if (distance.any() <= 0.001):
            print('event '+str(i)+' gives problems')
            print(distance[0])
        weighted_neighs = 1/distance[0]
        norm = np.sum(1/distance[0])
        labels = algo.label_train[closestn][0].tolist()
        for ll in range(0,len(labels)): 
            labels[ll] = int(labels[ll])
        labels = np.array(labels)
        ind_0 = np.where(labels == 0)
        ind_1 = np.where(labels == 1)
        ind_2 = np.where(labels == 2)
    
        neighs_0 = weighted_neighs[ind_0]
        neighs_1 = weighted_neighs[ind_1]
        neighs_2 = weighted_neighs[ind_2]

        nplus_NO[i] = np.sum(neighs_0)/norm
        nplus_NS[i] = np.sum(neighs_1)/norm
        nplus_REM[i] = np.sum(neighs_2)/norm
        
        if i == 5149:
            print(nplus_NO[i])
            print(nplus_NS[i])
            print(nplus_REM[i])
            print(labels)
            print(algo.label_test[i])
        
    
    with open('matrix_events_EOS-'+eos+'.txt','w+') as f:

        f.write('# HasNS_true \t HasREM_true \t f_0 \t f_1 \t f_2 \n')

        for i in range(0,total_events):

            f.write('%d \t\t %d \t\t %.4f \t %.4f \t %.4f \n'%(algo.label_NStrue[i],algo.label_REMtrue[i],nplus_NO[i],nplus_NS[i], nplus_REM[i]))
    
    return 

In [149]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    matrix(EOSdic[eos]['knn'],nneigh[0],eos)
    print('#'*60)

Doing EOS:  APR4_EPP
0.0
0.1693602487746655
0.8306397512253345
[2 2 2 2 2 1 2 1 2 2]
2.0


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:27: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  BHF_BBB2
0.0
0.5063130442293888
0.49368695577061117
[1 2 1 1 2 2 2 1 1 2]
2.0
############################################################
Doing EOS:  H4
0.0
0.2785828505991462
0.7214171494008537
[2 2 2 1 2 2 1 1 2 2]
2.0
############################################################
Doing EOS:  HQC18
1.0
0.0
0.0
[0 0 0 0 0 0 0 0 0 0]
0.0
############################################################
Doing EOS:  KDE0V
1.0
0.0
0.0
[0 0 0 0 0 0 0 0 0 0]
0.0
############################################################
Doing EOS:  KDE0V1
0.21496307812472298
0.7850369218752771
0.0
[1 0 1 1 1 1 1 1 1 0]
1.0
############################################################
Doing EOS:  MPA1
0.0
1.0
0.0
[1 1 1 1 1 1 1 1 1 1]
1.0


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:27: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  MS1_PP
0.0
1.0
0.0
[1 1 1 1 1 1 1 1 1 1]
1.0


...

############################################################
Doing EOS:  SLY2
1.0
0.0
0.0
[0 0 0 0 0 0 0 0 0 0]
0.0


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:27: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  SLY9
1.0
0.0
0.0
[0 0 0 0 0 0 0 0 0 0]
0.0


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/1517556471.py:27: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  SLY230A
1.0
0.0
0.0
[0 0 0 0 0 0 0 0 0 0]
0.0
############################################################


In [158]:
def matrix_forest(RFdic, ntrees,eos):
    
    total_events = len(RFdic['forest'].labels_test)
    
    predictions_trees =np.zeros(shape=(ntrees,len(RFdic['forest'].labels_test)))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(RFdic['forest'].data_test)
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        nplus_NS[i] = int(labels.count(1))/ntrees
        nplus_REM[i] = int(labels.count(2))/ntrees
        nplus_no[i] = int(labels.count(0))/ntrees
        
    with open('matrix_events_RF_EOS-'+eos+'.txt','w+') as f:

        f.write('# HasNS_true \t HasREM_true \t f_0 \t f_1 \t f_2 \n')

        for i in range(0,total_events):

            f.write('%d \t\t %d \t\t %.4f \t %.4f \t %.4f \n'%(RFdic['forest'].label_NS[i],RFdic['forest'].label_REM[i],nplus_no[i],nplus_NS[i], nplus_REM[i]))
    
        

In [159]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    matrix_forest(EOSdicRF[eos],50,eos)
    print('#'*60)

Doing EOS:  APR4_EPP
############################################################
Doing EOS:  BHF_BBB2
############################################################
Doing EOS:  H4
############################################################
Doing EOS:  HQC18
############################################################
Doing EOS:  KDE0V
############################################################
Doing EOS:  KDE0V1
############################################################
Doing EOS:  MPA1
############################################################
Doing EOS:  MS1_PP
############################################################
Doing EOS:  MS1B_PP
############################################################
Doing EOS:  RS
############################################################
Doing EOS:  SK255
############################################################
Doing EOS:  SK272
############################################################
Doing EOS:  SKI2
################################